# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from z3 import *
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import sys
import warnings
warnings.filterwarnings("ignore")
import json

# FIXED PARAMETERS

In [2]:
NUM_ZONES = 5

# Clustering

In [3]:
# returns kmeans clusters
def kmeans(X, number_clusters):
    kmeans = KMeans(n_clusters = number_clusters)
    cluster = kmeans.fit(X)
    return cluster

# Convex Hull

In [4]:
# returns convex hull of points associated with zones
def convex_hull(zone, points):
        '''
        Parameters
        ----------
        points : Array
            A set of points.
            
        Returns
        -------
        vertices : Points
            Arranges convex hull vertices in counter-clockwise orientation.
        '''    
        
        hull = ConvexHull(points)
        
        
        #plt.plot(points[:,0], points[:,1], 'o')
        #plt.ylim(0,120)
        #for simplex in hull.simplices:
        #    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
            
        simplices = hull.simplices
        
        vertices = []
        for index in hull.vertices:
            vertices.append((points[index][0], zone, points[index][1]))
        vertices.append((points[hull.vertices[0]][0], zone, points[hull.vertices[0]][1]))
        
        return vertices

# Cluster Boundary Acquisition

In [5]:
def get_cluster(dataframe, num_clusters):
    list_cluster = []
    for zone in range(NUM_ZONES):
        #plt.figure()
        mod_dataframe = pd.DataFrame()

        for i in range(len(dataframe)):
            if int(dataframe['Occupant\'s Zone'][i] == zone):
                mod_dataframe = mod_dataframe.append(dataframe.loc[i, ['Zone Arrival Time (Minute)', 'Stay Duration (Minute)']])

        features = np.empty([len(mod_dataframe), 2])
        features[:, 0:1] = mod_dataframe.loc[:, ['Zone Arrival Time (Minute)']].values
        features[:, 1:] = mod_dataframe.loc[:, ['Stay Duration (Minute)']].values
        cluster_model = kmeans(features, num_clusters)
        labels = cluster_model.labels_
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        for cluster in range(n_clusters):
            points = []
            vertices = []
            
            for k in range(len(labels)):
                if labels[k] == cluster:
                    points.append(features[k])
            
            if len(points) >= 3:
                try:
                    vertices = convex_hull(zone, points)       
                except:
                    pass
            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

    return list_cluster

# Clustering for House-A and Occupant-1

In [6]:
cleaned_duration_dataframe_house_A_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-1.csv")
cleaned_duration_dataframe_house_A_occ_1
clusters_house_A_occ_1 = get_cluster(cleaned_duration_dataframe_house_A_occ_1, 29)
# saving clusters to file
with open('../../data/shatter/KMeans_Clusters-House_A_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_1))

# Clustering for House-A and Occupant-2

In [7]:
cleaned_duration_dataframe_house_A_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-2.csv")
cleaned_duration_dataframe_house_A_occ_2
clusters_house_A_occ_2 = get_cluster(cleaned_duration_dataframe_house_A_occ_2, 15)
# saving clusters to file
with open('../../data/shatter/KMeans_Clusters-House_A_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_2))

# Clustering for House-B and Occupant-1

In [37]:
cleaned_duration_dataframe_house_B_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-1.csv")
cleaned_duration_dataframe_house_B_occ_1
clusters_house_B_occ_1 = get_cluster(cleaned_duration_dataframe_house_B_occ_1, 14)
# saving clusters to file
with open('../../data/shatter/KMeans_Clusters-House_B_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_1))

# Clustering for House-B and Occupant-2

In [8]:
cleaned_duration_dataframe_house_B_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-2.csv")
cleaned_duration_dataframe_house_B_occ_2
clusters_house_B_occ_2 = get_cluster(cleaned_duration_dataframe_house_B_occ_2, 10)
# saving clusters to file
with open('../../data/shatter/KMeans_Clusters-House_B_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_2))

# Range Claculation Function

In [9]:
def is_left( x, y, init_x, init_y, final_x, final_y):
     return ((final_x - init_x)*(y - init_y) - (final_y - init_y)*(x - init_x)) >= 0

def range_calculation(list_cluster):
    num_timeslots = 1440
    list_time_min = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]
    list_time_max = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]

    for i in range(len(list_cluster)):
        zone_id = list_cluster[i]["zone_id"]
        min_x_range = 1440
        max_x_range = 0

        ##################################################################
        ##################### Zone Constraints ###########################
        ##################################################################
        x = Int('x')
        y = Int('y')

        points = list_cluster[i]["points"]
        zone_constraints = []

        and_constraints = []
        for j in range(len(points) - 1):
            and_constraints.append(is_left(x, y, points[j][0], points[j][2], points[j + 1][0], points[j + 1][2]))

        zone_constraints.append(And(and_constraints))

        #print(zone_constraints)
        ####### Minimum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.minimize(x)
        o.check()
        
        min_x_range = int(str(o.model()[x]))

        ####### Maximum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.maximize(x)
        o.check()
        #print(o.model()[x])

        max_x_range = int(str(o.model()[x]))
        
        
        for j in range(min_x_range, max_x_range):
            ####### Minimum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.minimize(y)
            o.check()

            min_y_range = o.model()[y]
            if min_y_range == None:
                min_y_range = 0

            ####### Maximum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.maximize(y)
            o.check()

            max_y_range = o.model()[y]
            if max_y_range == None:
                max_y_range = 0
                
            list_time_min[zone_id][j].append(int(str(min_y_range)))
            list_time_max[zone_id][j].append(int(str(max_y_range)))
    return list_time_min, list_time_max

# Attack Scheduling Function

In [19]:
def attack_scheduling(list_cluster, num_timeslots, start_time, final_time):
    list_time_min, list_time_max = range_calculation(list_cluster)
    num_timeslots = interval = 10

    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))

    for init_time in range(start_time, final_time, interval):
        cost = [Int( 'cost_' + str(i)) for i in range(NUM_ZONES)]
        zones = [Int( 'zones_' + str(i)) for i in range(NUM_ZONES)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(num_timeslots)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(num_timeslots)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(NUM_ZONES):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(1, NUM_ZONES):
            if list_time_min[z][init_time] == [] or (len(list_time_max[z][init_time]) == 1 and list_time_max[z][init_time][0] == 0):
                o.add(Implies(schedule[0] != prev_schedule, schedule[t] != z))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                if list_time_min[z][init_time + t] == [] or (len(list_time_max[z][init_time + t]) == 1 and list_time_max[z][init_time + t][0] == 0):
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(NUM_ZONES):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        #o.add(schedule[0] == 3)

        o.maximize(total_cost)

        print(o.check())
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))

        print("Ulimate Cost", init_time, ultimate_cost)

        print("Num_timeslot", num_timeslots, "Optimal_cost", o.model()[total_cost])

        for t in range(num_timeslots):
            print("schedule_" + str(init_time + t), o.model()[schedule[t]])
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        for t in range(num_timeslots):
            print("stay_" + str(init_time + t), o.model()[stay[t]])

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
    return final_schedule

# Greedy Scheduling Function

In [10]:
def greedy_schedule(list_cluster, num_timeslots):
    current_timeslot = 0
    final_timeslot = 0

    alternate_schedule = np.zeros((num_timeslots))
    list_time_min, list_time_max = range_calculation(list_cluster)
    while final_timeslot <= 1439:
        for zone in [3, 4, 2, 1, 0]:
            max_list = list_time_max[zone][current_timeslot]
            for m in range(len(max_list)):
                max_list[m] = int(str(max_list[m]))
            if max_list == []:
                if zone == 0:
                    print("Critical", current_timeslot)
                    final_timeslot = current_timeslot + 1
                    alternate_schedule[current_timeslot: final_timeslot] = 0
                    current_timeslot = final_timeslot
                continue
            max_val = max(max_list)
            if max_val == 0:
                final_timeslot = current_timeslot + 1
                alternate_schedule[current_timeslot: final_timeslot] = 0
                current_timeslot = final_timeslot
            if max_val != 0:
                final_timeslot = current_timeslot + max_val
                print("Here", current_timeslot, final_timeslot)
                alternate_schedule[current_timeslot: final_timeslot] = zone
                print("selected zone", zone)
                current_timeslot = final_timeslot
                break
    return alternate_schedule

# Variable Parameters

In [11]:
num_timeslots = interval = 10
start_time = 0
final_time = 1440

# House-A Occupant-1 Attack Schedule

In [22]:
schedule_house_A_occ_1 = attack_scheduling(clusters_house_A_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-A_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 40
Ulimate Cost 0 40
Num_timeslot 10 Optimal_cost 40
schedule_0 3
schedule_1 3
schedule_2 3
schedule_3 3
schedule_4 3
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 36
Ulimate Cost 10 76
Num_timeslot 10 Optimal_cost 36
schedule_10 3
schedule_11 3
schedule_12 3
schedule_13 3
schedule_14 0
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 1
stay_15 1
stay_16 2
stay_17 3
stay_18 4
stay_19 5
sat
20 22
Ulimate Cost 20 98
Num_timeslot 10 Optimal_cost 22
schedule_20 3
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 6
stay_21 1
stay_22 2
stay_23 3
stay_24 4
stay_25 5
stay_26 6
stay_27 7
stay_28 8
stay_29 9
sat
30 24
Ulimate Cost 30 122
Num_timeslot 10 Optimal_cost 24
schedule_30 2
schedule_31 2
schedule_32 2
schedule_3

sat
260 10
Ulimate Cost 260 564
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 91
stay_261 92
stay_262 93
stay_263 94
stay_264 95
stay_265 96
stay_266 97
stay_267 98
stay_268 99
stay_269 100
sat
270 10
Ulimate Cost 270 574
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 101
stay_271 102
stay_272 103
stay_273 104
stay_274 105
stay_275 106
stay_276 107
stay_277 108
stay_278 109
stay_279 110
sat
280 10
Ulimate Cost 280 584
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 111
stay_281 112
stay_282 113
stay_283 114
stay_284 115
stay_285 116
stay_286 117
stay_287 118
sta

sat
500 10
Ulimate Cost 500 804
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 331
stay_501 332
stay_502 333
stay_503 334
stay_504 335
stay_505 336
stay_506 337
stay_507 338
stay_508 339
stay_509 340
sat
510 10
Ulimate Cost 510 814
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 341
stay_511 342
stay_512 343
stay_513 344
stay_514 345
stay_515 346
stay_516 347
stay_517 348
stay_518 349
stay_519 350
sat
520 10
Ulimate Cost 520 824
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 351
stay_521 352
stay_522 353
stay_523 354
stay_524 355
stay_525 356
stay_526 357
stay_52

sat
750 40
Ulimate Cost 750 1540
Num_timeslot 10 Optimal_cost 40
schedule_750 3
schedule_751 3
schedule_752 3
schedule_753 3
schedule_754 3
schedule_755 3
schedule_756 3
schedule_757 3
schedule_758 3
schedule_759 3
stay_750 40
stay_751 41
stay_752 42
stay_753 43
stay_754 44
stay_755 45
stay_756 46
stay_757 47
stay_758 48
stay_759 49
sat
760 40
Ulimate Cost 760 1580
Num_timeslot 10 Optimal_cost 40
schedule_760 3
schedule_761 3
schedule_762 3
schedule_763 3
schedule_764 3
schedule_765 3
schedule_766 3
schedule_767 3
schedule_768 3
schedule_769 3
stay_760 50
stay_761 51
stay_762 52
stay_763 53
stay_764 54
stay_765 55
stay_766 56
stay_767 57
stay_768 58
stay_769 59
sat
770 40
Ulimate Cost 770 1620
Num_timeslot 10 Optimal_cost 40
schedule_770 3
schedule_771 3
schedule_772 3
schedule_773 3
schedule_774 3
schedule_775 3
schedule_776 3
schedule_777 3
schedule_778 3
schedule_779 3
stay_770 60
stay_771 61
stay_772 62
stay_773 63
stay_774 64
stay_775 65
stay_776 66
stay_777 67
stay_778 68
stay_77

sat
1000 40
Ulimate Cost 1000 2489
Num_timeslot 10 Optimal_cost 40
schedule_1000 3
schedule_1001 3
schedule_1002 3
schedule_1003 3
schedule_1004 3
schedule_1005 3
schedule_1006 3
schedule_1007 3
schedule_1008 3
schedule_1009 3
stay_1000 140
stay_1001 141
stay_1002 142
stay_1003 143
stay_1004 144
stay_1005 145
stay_1006 146
stay_1007 147
stay_1008 148
stay_1009 149
sat
1010 40
Ulimate Cost 1010 2529
Num_timeslot 10 Optimal_cost 40
schedule_1010 3
schedule_1011 3
schedule_1012 3
schedule_1013 3
schedule_1014 3
schedule_1015 3
schedule_1016 3
schedule_1017 3
schedule_1018 3
schedule_1019 3
stay_1010 150
stay_1011 151
stay_1012 152
stay_1013 153
stay_1014 154
stay_1015 155
stay_1016 156
stay_1017 157
stay_1018 158
stay_1019 159
sat
1020 40
Ulimate Cost 1020 2569
Num_timeslot 10 Optimal_cost 40
schedule_1020 3
schedule_1021 3
schedule_1022 3
schedule_1023 3
schedule_1024 3
schedule_1025 3
schedule_1026 3
schedule_1027 3
schedule_1028 3
schedule_1029 3
stay_1020 160
stay_1021 161
stay_1022 1

sat
1230 40
Ulimate Cost 1230 3409
Num_timeslot 10 Optimal_cost 40
schedule_1230 3
schedule_1231 3
schedule_1232 3
schedule_1233 3
schedule_1234 3
schedule_1235 3
schedule_1236 3
schedule_1237 3
schedule_1238 3
schedule_1239 3
stay_1230 370
stay_1231 371
stay_1232 372
stay_1233 373
stay_1234 374
stay_1235 375
stay_1236 376
stay_1237 377
stay_1238 378
stay_1239 379
sat
1240 40
Ulimate Cost 1240 3449
Num_timeslot 10 Optimal_cost 40
schedule_1240 3
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 380
stay_1241 381
stay_1242 382
stay_1243 383
stay_1244 384
stay_1245 385
stay_1246 386
stay_1247 387
stay_1248 388
stay_1249 389
sat
1250 40
Ulimate Cost 1250 3489
Num_timeslot 10 Optimal_cost 40
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 3
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 390
stay_1251 391
stay_1252 3

# House-A Occupant-1 Greedy Schedule

In [23]:
greedy_schedule_house_A_occ_1 = greedy_schedule(clusters_house_A_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-A_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 18
selected zone 3
Here 18 22
selected zone 3
Here 22 148
selected zone 2
Here 148 154
selected zone 4
Here 154 161
selected zone 4
Here 161 168
selected zone 4
Here 168 639
selected zone 1
Here 639 716
selected zone 3
Here 716 801
selected zone 3
Here 801 828
selected zone 4
Here 828 833
selected zone 3
Here 833 839
selected zone 3
Here 839 845
selected zone 3
Here 845 851
selected zone 3
Here 851 857
selected zone 3
Here 857 860
selected zone 3
Here 860 862
selected zone 3
Here 862 898
selected zone 4
Here 898 923
selected zone 3
Here 923 926
selected zone 4
Here 926 969
selected zone 3
Here 969 1005
selected zone 3
Here 1005 1044
selected zone 3
Here 1044 1101
selected zone 3
Here 1101 1139
selected zone 4
Here 1139 1186
selected zone 3
Here 1186 1269
selected zone 3
Here 1269 1272
selected zone 4
Here 1272 1283
selected zone 3
Here 1283 1332
selected zone 3
Here 1332 1365
selected zone 3
Here 1365 1386
selected zone 1
Here 1386 1393
selected zone 3
Here 1393 1400
selected zo

# House-A Occupant-2 Attack Schedule

In [24]:
schedule_house_A_occ_2 = attack_scheduling(clusters_house_A_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-A_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 36
Ulimate Cost 0 36
Num_timeslot 10 Optimal_cost 36
schedule_0 0
schedule_1 3
schedule_2 3
schedule_3 3
schedule_4 3
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 1
stay_2 2
stay_3 3
stay_4 4
stay_5 5
stay_6 6
stay_7 7
stay_8 8
stay_9 9
sat
10 40
Ulimate Cost 10 76
Num_timeslot 10 Optimal_cost 40
schedule_10 3
schedule_11 3
schedule_12 3
schedule_13 3
schedule_14 3
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 10
stay_11 11
stay_12 12
stay_13 13
stay_14 14
stay_15 15
stay_16 16
stay_17 17
stay_18 18
stay_19 19
sat
20 40
Ulimate Cost 20 116
Num_timeslot 10 Optimal_cost 40
schedule_20 3
schedule_21 3
schedule_22 3
schedule_23 3
schedule_24 3
schedule_25 3
schedule_26 3
schedule_27 3
schedule_28 3
schedule_29 3
stay_20 20
stay_21 21
stay_22 22
stay_23 23
stay_24 24
stay_25 25
stay_26 26
stay_27 27
stay_28 28
stay_29 29
sat
30 40
Ulimate Cost 30 156
Num_timeslot 10 Optimal_cost 40
schedule_30 3
schedule_31 3
schedule

sat
250 40
Ulimate Cost 250 1036
Num_timeslot 10 Optimal_cost 40
schedule_250 3
schedule_251 3
schedule_252 3
schedule_253 3
schedule_254 3
schedule_255 3
schedule_256 3
schedule_257 3
schedule_258 3
schedule_259 3
stay_250 250
stay_251 251
stay_252 252
stay_253 253
stay_254 254
stay_255 255
stay_256 256
stay_257 257
stay_258 258
stay_259 259
sat
260 40
Ulimate Cost 260 1076
Num_timeslot 10 Optimal_cost 40
schedule_260 3
schedule_261 3
schedule_262 3
schedule_263 3
schedule_264 3
schedule_265 3
schedule_266 3
schedule_267 3
schedule_268 3
schedule_269 3
stay_260 260
stay_261 261
stay_262 262
stay_263 263
stay_264 264
stay_265 265
stay_266 266
stay_267 267
stay_268 268
stay_269 269
sat
270 40
Ulimate Cost 270 1116
Num_timeslot 10 Optimal_cost 40
schedule_270 3
schedule_271 3
schedule_272 3
schedule_273 3
schedule_274 3
schedule_275 3
schedule_276 3
schedule_277 3
schedule_278 3
schedule_279 3
stay_270 270
stay_271 271
stay_272 272
stay_273 273
stay_274 274
stay_275 275
stay_276 276
stay

sat
490 40
Ulimate Cost 490 1996
Num_timeslot 10 Optimal_cost 40
schedule_490 3
schedule_491 3
schedule_492 3
schedule_493 3
schedule_494 3
schedule_495 3
schedule_496 3
schedule_497 3
schedule_498 3
schedule_499 3
stay_490 490
stay_491 491
stay_492 492
stay_493 493
stay_494 494
stay_495 495
stay_496 496
stay_497 497
stay_498 498
stay_499 499
sat
500 40
Ulimate Cost 500 2036
Num_timeslot 10 Optimal_cost 40
schedule_500 3
schedule_501 3
schedule_502 3
schedule_503 3
schedule_504 3
schedule_505 3
schedule_506 3
schedule_507 3
schedule_508 3
schedule_509 3
stay_500 500
stay_501 501
stay_502 502
stay_503 503
stay_504 504
stay_505 505
stay_506 506
stay_507 507
stay_508 508
stay_509 509
sat
510 40
Ulimate Cost 510 2076
Num_timeslot 10 Optimal_cost 40
schedule_510 3
schedule_511 3
schedule_512 3
schedule_513 3
schedule_514 3
schedule_515 3
schedule_516 3
schedule_517 3
schedule_518 3
schedule_519 3
stay_510 510
stay_511 511
stay_512 512
stay_513 513
stay_514 514
stay_515 515
stay_516 516
stay

sat
730 40
Ulimate Cost 730 2956
Num_timeslot 10 Optimal_cost 40
schedule_730 3
schedule_731 3
schedule_732 3
schedule_733 3
schedule_734 3
schedule_735 3
schedule_736 3
schedule_737 3
schedule_738 3
schedule_739 3
stay_730 730
stay_731 731
stay_732 732
stay_733 733
stay_734 734
stay_735 735
stay_736 736
stay_737 737
stay_738 738
stay_739 739
sat
740 40
Ulimate Cost 740 2996
Num_timeslot 10 Optimal_cost 40
schedule_740 3
schedule_741 3
schedule_742 3
schedule_743 3
schedule_744 3
schedule_745 3
schedule_746 3
schedule_747 3
schedule_748 3
schedule_749 3
stay_740 740
stay_741 741
stay_742 742
stay_743 743
stay_744 744
stay_745 745
stay_746 746
stay_747 747
stay_748 748
stay_749 749
sat
750 40
Ulimate Cost 750 3036
Num_timeslot 10 Optimal_cost 40
schedule_750 3
schedule_751 3
schedule_752 3
schedule_753 3
schedule_754 3
schedule_755 3
schedule_756 3
schedule_757 3
schedule_758 3
schedule_759 3
stay_750 750
stay_751 751
stay_752 752
stay_753 753
stay_754 754
stay_755 755
stay_756 756
stay

sat
970 40
Ulimate Cost 970 3916
Num_timeslot 10 Optimal_cost 40
schedule_970 3
schedule_971 3
schedule_972 3
schedule_973 3
schedule_974 3
schedule_975 3
schedule_976 3
schedule_977 3
schedule_978 3
schedule_979 3
stay_970 970
stay_971 971
stay_972 972
stay_973 973
stay_974 974
stay_975 975
stay_976 976
stay_977 977
stay_978 978
stay_979 979
sat
980 40
Ulimate Cost 980 3956
Num_timeslot 10 Optimal_cost 40
schedule_980 3
schedule_981 3
schedule_982 3
schedule_983 3
schedule_984 3
schedule_985 3
schedule_986 3
schedule_987 3
schedule_988 3
schedule_989 3
stay_980 980
stay_981 981
stay_982 982
stay_983 983
stay_984 984
stay_985 985
stay_986 986
stay_987 987
stay_988 988
stay_989 989
sat
990 40
Ulimate Cost 990 3996
Num_timeslot 10 Optimal_cost 40
schedule_990 3
schedule_991 3
schedule_992 3
schedule_993 3
schedule_994 3
schedule_995 3
schedule_996 3
schedule_997 3
schedule_998 3
schedule_999 3
stay_990 990
stay_991 991
stay_992 992
stay_993 993
stay_994 994
stay_995 995
stay_996 996
stay

sat
1190 40
Ulimate Cost 1190 4796
Num_timeslot 10 Optimal_cost 40
schedule_1190 3
schedule_1191 3
schedule_1192 3
schedule_1193 3
schedule_1194 3
schedule_1195 3
schedule_1196 3
schedule_1197 3
schedule_1198 3
schedule_1199 3
stay_1190 1190
stay_1191 1191
stay_1192 1192
stay_1193 1193
stay_1194 1194
stay_1195 1195
stay_1196 1196
stay_1197 1197
stay_1198 1198
stay_1199 1199
sat
1200 40
Ulimate Cost 1200 4836
Num_timeslot 10 Optimal_cost 40
schedule_1200 3
schedule_1201 3
schedule_1202 3
schedule_1203 3
schedule_1204 3
schedule_1205 3
schedule_1206 3
schedule_1207 3
schedule_1208 3
schedule_1209 3
stay_1200 1200
stay_1201 1201
stay_1202 1202
stay_1203 1203
stay_1204 1204
stay_1205 1205
stay_1206 1206
stay_1207 1207
stay_1208 1208
stay_1209 1209
sat
1210 40
Ulimate Cost 1210 4876
Num_timeslot 10 Optimal_cost 40
schedule_1210 3
schedule_1211 3
schedule_1212 3
schedule_1213 3
schedule_1214 3
schedule_1215 3
schedule_1216 3
schedule_1217 3
schedule_1218 3
schedule_1219 3
stay_1210 1210
stay

sat
1410 40
Ulimate Cost 1410 5676
Num_timeslot 10 Optimal_cost 40
schedule_1410 3
schedule_1411 3
schedule_1412 3
schedule_1413 3
schedule_1414 3
schedule_1415 3
schedule_1416 3
schedule_1417 3
schedule_1418 3
schedule_1419 3
stay_1410 1410
stay_1411 1411
stay_1412 1412
stay_1413 1413
stay_1414 1414
stay_1415 1415
stay_1416 1416
stay_1417 1417
stay_1418 1418
stay_1419 1419
sat
1420 40
Ulimate Cost 1420 5716
Num_timeslot 10 Optimal_cost 40
schedule_1420 3
schedule_1421 3
schedule_1422 3
schedule_1423 3
schedule_1424 3
schedule_1425 3
schedule_1426 3
schedule_1427 3
schedule_1428 3
schedule_1429 3
stay_1420 1420
stay_1421 1421
stay_1422 1422
stay_1423 1423
stay_1424 1424
stay_1425 1425
stay_1426 1426
stay_1427 1427
stay_1428 1428
stay_1429 1429
sat
1430 40
Ulimate Cost 1430 5756
Num_timeslot 10 Optimal_cost 40
schedule_1430 3
schedule_1431 3
schedule_1432 3
schedule_1433 3
schedule_1434 3
schedule_1435 3
schedule_1436 3
schedule_1437 3
schedule_1438 3
schedule_1439 3
stay_1430 1430
stay

# House-A Occupant-2 Greedy Schedule

In [25]:
greedy_schedule_house_A_occ_2 = greedy_schedule(clusters_house_A_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-A_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 1
selected zone 3
Here 2 45
selected zone 2
Here 45 81
selected zone 3
Here 81 117
selected zone 3
Here 117 157
selected zone 3
Here 157 178
selected zone 4
Here 178 207
selected zone 2
Here 207 227
selected zone 2
Here 227 231
selected zone 4
Here 231 236
selected zone 4
Here 236 242
selected zone 4
Here 242 250
selected zone 4
Here 250 260
selected zone 4
Here 260 272
selected zone 4
Here 272 287
selected zone 4
Here 287 306
selected zone 4
Here 306 330
selected zone 4
Here 330 360
selected zone 4
Here 360 635
selected zone 1
Here 635 690
selected zone 2
Here 690 749
selected zone 2
Here 749 883
selected zone 2
Here 883 899
selected zone 4
Here 899 919
selected zone 4
Here 919 933
selected zone 4
Here 933 942
selected zone 4
Here 942 957
selected zone 1
Here 957 1437
selected zone 0
Critical 1437
Critical 1438
Critical 1439


# House-B Occupant-1 Attack Schedule

In [26]:
schedule_house_B_occ_1 = attack_scheduling(clusters_house_B_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-B_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 23
Ulimate Cost 0 23
Num_timeslot 10 Optimal_cost 23
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 4
schedule_6 4
schedule_7 4
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 1
stay_6 2
stay_7 3
stay_8 1
stay_9 2
sat
10 30
Ulimate Cost 10 53
Num_timeslot 10 Optimal_cost 30
schedule_10 4
schedule_11 4
schedule_12 4
schedule_13 4
schedule_14 4
schedule_15 4
schedule_16 4
schedule_17 4
schedule_18 4
schedule_19 4
stay_10 1
stay_11 2
stay_12 3
stay_13 4
stay_14 5
stay_15 6
stay_16 7
stay_17 8
stay_18 9
stay_19 10
sat
20 30
Ulimate Cost 20 83
Num_timeslot 10 Optimal_cost 30
schedule_20 4
schedule_21 4
schedule_22 4
schedule_23 4
schedule_24 4
schedule_25 4
schedule_26 4
schedule_27 4
schedule_28 4
schedule_29 4
stay_20 11
stay_21 12
stay_22 13
stay_23 14
stay_24 15
stay_25 16
stay_26 17
stay_27 18
stay_28 19
stay_29 20
sat
30 30
Ulimate Cost 30 113
Num_timeslot 10 Optimal_cost 30
schedule_30 4
schedule_31 4
schedule_32 4
sche

sat
250 30
Ulimate Cost 250 773
Num_timeslot 10 Optimal_cost 30
schedule_250 4
schedule_251 4
schedule_252 4
schedule_253 4
schedule_254 4
schedule_255 4
schedule_256 4
schedule_257 4
schedule_258 4
schedule_259 4
stay_250 241
stay_251 242
stay_252 243
stay_253 244
stay_254 245
stay_255 246
stay_256 247
stay_257 248
stay_258 249
stay_259 250
sat
260 30
Ulimate Cost 260 803
Num_timeslot 10 Optimal_cost 30
schedule_260 4
schedule_261 4
schedule_262 4
schedule_263 4
schedule_264 4
schedule_265 4
schedule_266 4
schedule_267 4
schedule_268 4
schedule_269 4
stay_260 251
stay_261 252
stay_262 253
stay_263 254
stay_264 255
stay_265 256
stay_266 257
stay_267 258
stay_268 259
stay_269 260
sat
270 30
Ulimate Cost 270 833
Num_timeslot 10 Optimal_cost 30
schedule_270 4
schedule_271 4
schedule_272 4
schedule_273 4
schedule_274 4
schedule_275 4
schedule_276 4
schedule_277 4
schedule_278 4
schedule_279 4
stay_270 261
stay_271 262
stay_272 263
stay_273 264
stay_274 265
stay_275 266
stay_276 267
stay_27

sat
490 30
Ulimate Cost 490 1493
Num_timeslot 10 Optimal_cost 30
schedule_490 4
schedule_491 4
schedule_492 4
schedule_493 4
schedule_494 4
schedule_495 4
schedule_496 4
schedule_497 4
schedule_498 4
schedule_499 4
stay_490 481
stay_491 482
stay_492 483
stay_493 484
stay_494 485
stay_495 486
stay_496 487
stay_497 488
stay_498 489
stay_499 490
sat
500 30
Ulimate Cost 500 1523
Num_timeslot 10 Optimal_cost 30
schedule_500 4
schedule_501 4
schedule_502 4
schedule_503 4
schedule_504 4
schedule_505 4
schedule_506 4
schedule_507 4
schedule_508 4
schedule_509 4
stay_500 491
stay_501 492
stay_502 493
stay_503 494
stay_504 495
stay_505 496
stay_506 497
stay_507 498
stay_508 499
stay_509 500
sat
510 30
Ulimate Cost 510 1553
Num_timeslot 10 Optimal_cost 30
schedule_510 4
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 4
stay_510 501
stay_511 502
stay_512 503
stay_513 504
stay_514 505
stay_515 506
stay_516 507
stay

sat
730 30
Ulimate Cost 730 2213
Num_timeslot 10 Optimal_cost 30
schedule_730 4
schedule_731 4
schedule_732 4
schedule_733 4
schedule_734 4
schedule_735 4
schedule_736 4
schedule_737 4
schedule_738 4
schedule_739 4
stay_730 721
stay_731 722
stay_732 723
stay_733 724
stay_734 725
stay_735 726
stay_736 727
stay_737 728
stay_738 729
stay_739 730
sat
740 30
Ulimate Cost 740 2243
Num_timeslot 10 Optimal_cost 30
schedule_740 4
schedule_741 4
schedule_742 4
schedule_743 4
schedule_744 4
schedule_745 4
schedule_746 4
schedule_747 4
schedule_748 4
schedule_749 4
stay_740 731
stay_741 732
stay_742 733
stay_743 734
stay_744 735
stay_745 736
stay_746 737
stay_747 738
stay_748 739
stay_749 740
sat
750 30
Ulimate Cost 750 2273
Num_timeslot 10 Optimal_cost 30
schedule_750 4
schedule_751 4
schedule_752 4
schedule_753 4
schedule_754 4
schedule_755 4
schedule_756 4
schedule_757 4
schedule_758 4
schedule_759 4
stay_750 741
stay_751 742
stay_752 743
stay_753 744
stay_754 745
stay_755 746
stay_756 747
stay

sat
970 30
Ulimate Cost 970 2933
Num_timeslot 10 Optimal_cost 30
schedule_970 4
schedule_971 4
schedule_972 4
schedule_973 4
schedule_974 4
schedule_975 4
schedule_976 4
schedule_977 4
schedule_978 4
schedule_979 4
stay_970 961
stay_971 962
stay_972 963
stay_973 964
stay_974 965
stay_975 966
stay_976 967
stay_977 968
stay_978 969
stay_979 970
sat
980 30
Ulimate Cost 980 2963
Num_timeslot 10 Optimal_cost 30
schedule_980 4
schedule_981 4
schedule_982 4
schedule_983 4
schedule_984 4
schedule_985 4
schedule_986 4
schedule_987 4
schedule_988 4
schedule_989 4
stay_980 971
stay_981 972
stay_982 973
stay_983 974
stay_984 975
stay_985 976
stay_986 977
stay_987 978
stay_988 979
stay_989 980
sat
990 30
Ulimate Cost 990 2993
Num_timeslot 10 Optimal_cost 30
schedule_990 4
schedule_991 4
schedule_992 4
schedule_993 4
schedule_994 4
schedule_995 4
schedule_996 4
schedule_997 4
schedule_998 4
schedule_999 4
stay_990 981
stay_991 982
stay_992 983
stay_993 984
stay_994 985
stay_995 986
stay_996 987
stay

sat
1200 30
Ulimate Cost 1200 3623
Num_timeslot 10 Optimal_cost 30
schedule_1200 4
schedule_1201 4
schedule_1202 4
schedule_1203 4
schedule_1204 4
schedule_1205 4
schedule_1206 4
schedule_1207 4
schedule_1208 4
schedule_1209 4
stay_1200 1191
stay_1201 1192
stay_1202 1193
stay_1203 1194
stay_1204 1195
stay_1205 1196
stay_1206 1197
stay_1207 1198
stay_1208 1199
stay_1209 1200
sat
1210 30
Ulimate Cost 1210 3653
Num_timeslot 10 Optimal_cost 30
schedule_1210 4
schedule_1211 4
schedule_1212 4
schedule_1213 4
schedule_1214 4
schedule_1215 4
schedule_1216 4
schedule_1217 4
schedule_1218 4
schedule_1219 4
stay_1210 1201
stay_1211 1202
stay_1212 1203
stay_1213 1204
stay_1214 1205
stay_1215 1206
stay_1216 1207
stay_1217 1208
stay_1218 1209
stay_1219 1210
sat
1220 30
Ulimate Cost 1220 3683
Num_timeslot 10 Optimal_cost 30
schedule_1220 4
schedule_1221 4
schedule_1222 4
schedule_1223 4
schedule_1224 4
schedule_1225 4
schedule_1226 4
schedule_1227 4
schedule_1228 4
schedule_1229 4
stay_1220 1211
stay

sat
1420 30
Ulimate Cost 1420 4283
Num_timeslot 10 Optimal_cost 30
schedule_1420 4
schedule_1421 4
schedule_1422 4
schedule_1423 4
schedule_1424 4
schedule_1425 4
schedule_1426 4
schedule_1427 4
schedule_1428 4
schedule_1429 4
stay_1420 1411
stay_1421 1412
stay_1422 1413
stay_1423 1414
stay_1424 1415
stay_1425 1416
stay_1426 1417
stay_1427 1418
stay_1428 1419
stay_1429 1420
sat
1430 30
Ulimate Cost 1430 4313
Num_timeslot 10 Optimal_cost 30
schedule_1430 4
schedule_1431 4
schedule_1432 4
schedule_1433 4
schedule_1434 4
schedule_1435 4
schedule_1436 4
schedule_1437 4
schedule_1438 4
schedule_1439 4
stay_1430 1421
stay_1431 1422
stay_1432 1423
stay_1433 1424
stay_1434 1425
stay_1435 1426
stay_1436 1427
stay_1437 1428
stay_1438 1429
stay_1439 1430


# House-B Occupant-1 Greedy Schedule

In [27]:
greedy_schedule_house_B_occ_1 = greedy_schedule(clusters_house_B_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-B_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 61
selected zone 2
Here 61 62
selected zone 4
Here 62 63
selected zone 4
Here 63 64
selected zone 4
Here 64 65
selected zone 4
Here 65 66
selected zone 4
Here 66 67
selected zone 4
Here 67 68
selected zone 4
Here 68 69
selected zone 4
Here 69 70
selected zone 4
Here 70 71
selected zone 4
Here 71 73
selected zone 4
Here 73 75
selected zone 4
Here 75 77
selected zone 4
Here 77 79
selected zone 4
Here 79 81
selected zone 4
Here 81 83
selected zone 4
Here 83 85
selected zone 4
Here 85 87
selected zone 4
Here 87 89
selected zone 4
Here 89 91
selected zone 4
Here 91 93
selected zone 4
Here 93 95
selected zone 4
Here 95 97
selected zone 4
Here 97 99
selected zone 4
Here 99 100
selected zone 4
Here 100 101
selected zone 4
Here 101 102
selected zone 4
Here 102 103
selected zone 4
Here 103 104
selected zone 4
Here 104 105
selected zone 4
Here 105 106
selected zone 4
Here 106 107
selected zone 4
Here 107 108
selected zone 4
Here 108 109
selected zone 4
Here 109 110
selected zone 4
Here 110

# House-B Occupant-2 Attack Schedule

In [28]:
schedule_house_B_occ_2 = attack_scheduling(clusters_house_B_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-B_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 20
Ulimate Cost 0 20
Num_timeslot 10 Optimal_cost 20
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 2
schedule_7 2
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 20
Ulimate Cost 10 40
Num_timeslot 10 Optimal_cost 20
schedule_10 2
schedule_11 2
schedule_12 2
schedule_13 2
schedule_14 2
schedule_15 2
schedule_16 2
schedule_17 2
schedule_18 2
schedule_19 2
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 22
Ulimate Cost 20 62
Num_timeslot 10 Optimal_cost 22
schedule_20 2
schedule_21 2
schedule_22 4
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 4
stay_20 21
stay_21 22
stay_22 1
stay_23 1
stay_24 2
stay_25 3
stay_26 4
stay_27 5
stay_28 6
stay_29 1
sat
30 22
Ulimate Cost 30 84
Num_timeslot 10 Optimal_cost 22
schedule_30 2
schedule_31 2
schedule_32 2
sch

sat
260 10
Ulimate Cost 260 448
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 32
stay_261 33
stay_262 34
stay_263 35
stay_264 36
stay_265 37
stay_266 38
stay_267 39
stay_268 40
stay_269 41
sat
270 10
Ulimate Cost 270 458
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 42
stay_271 43
stay_272 44
stay_273 45
stay_274 46
stay_275 47
stay_276 48
stay_277 49
stay_278 50
stay_279 51
sat
280 10
Ulimate Cost 280 468
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 52
stay_281 53
stay_282 54
stay_283 55
stay_284 56
stay_285 57
stay_286 58
stay_287 59
stay_288 60
stay_289 6

sat
510 20
Ulimate Cost 510 755
Num_timeslot 10 Optimal_cost 20
schedule_510 2
schedule_511 2
schedule_512 2
schedule_513 2
schedule_514 2
schedule_515 2
schedule_516 2
schedule_517 2
schedule_518 2
schedule_519 2
stay_510 48
stay_511 49
stay_512 50
stay_513 51
stay_514 52
stay_515 53
stay_516 54
stay_517 55
stay_518 56
stay_519 57
sat
520 20
Ulimate Cost 520 775
Num_timeslot 10 Optimal_cost 20
schedule_520 2
schedule_521 2
schedule_522 2
schedule_523 2
schedule_524 2
schedule_525 2
schedule_526 2
schedule_527 2
schedule_528 2
schedule_529 2
stay_520 58
stay_521 59
stay_522 60
stay_523 61
stay_524 62
stay_525 63
stay_526 64
stay_527 65
stay_528 66
stay_529 67
sat
530 20
Ulimate Cost 530 795
Num_timeslot 10 Optimal_cost 20
schedule_530 2
schedule_531 2
schedule_532 2
schedule_533 2
schedule_534 2
schedule_535 2
schedule_536 2
schedule_537 2
schedule_538 2
schedule_539 2
stay_530 68
stay_531 69
stay_532 70
stay_533 71
stay_534 72
stay_535 73
stay_536 74
stay_537 75
stay_538 76
stay_539 7

sat
760 30
Ulimate Cost 760 1471
Num_timeslot 10 Optimal_cost 30
schedule_760 4
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 4
schedule_765 4
schedule_766 4
schedule_767 4
schedule_768 4
schedule_769 4
stay_760 6
stay_761 7
stay_762 8
stay_763 9
stay_764 10
stay_765 11
stay_766 12
stay_767 13
stay_768 14
stay_769 15
sat
770 28
Ulimate Cost 770 1499
Num_timeslot 10 Optimal_cost 28
schedule_770 1
schedule_771 4
schedule_772 4
schedule_773 4
schedule_774 4
schedule_775 4
schedule_776 4
schedule_777 4
schedule_778 4
schedule_779 4
stay_770 1
stay_771 1
stay_772 2
stay_773 3
stay_774 4
stay_775 5
stay_776 6
stay_777 7
stay_778 8
stay_779 9
sat
780 28
Ulimate Cost 780 1527
Num_timeslot 10 Optimal_cost 28
schedule_780 4
schedule_781 4
schedule_782 4
schedule_783 4
schedule_784 4
schedule_785 4
schedule_786 4
schedule_787 1
schedule_788 4
schedule_789 4
stay_780 10
stay_781 11
stay_782 12
stay_783 13
stay_784 14
stay_785 15
stay_786 16
stay_787 1
stay_788 1
stay_789 2
sat
790 28
U

sat
1010 0
Ulimate Cost 1010 1952
Num_timeslot 10 Optimal_cost 0
schedule_1010 0
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 2
stay_1011 3
stay_1012 4
stay_1013 5
stay_1014 6
stay_1015 7
stay_1016 8
stay_1017 9
stay_1018 10
stay_1019 11
sat
1020 0
Ulimate Cost 1020 1952
Num_timeslot 10 Optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 12
stay_1021 13
stay_1022 14
stay_1023 15
stay_1024 16
stay_1025 17
stay_1026 18
stay_1027 19
stay_1028 20
stay_1029 21
sat
1030 0
Ulimate Cost 1030 1952
Num_timeslot 10 Optimal_cost 0
schedule_1030 0
schedule_1031 0
schedule_1032 0
schedule_1033 0
schedule_1034 0
schedule_1035 0
schedule_1036 0
schedule_1037 0
schedule_1038 0
schedule_1039 0
stay_1030 22
stay_1031 23
stay_1032 24
stay_1033 25
stay_1034 26
stay_103

sat
1250 20
Ulimate Cost 1250 2263
Num_timeslot 10 Optimal_cost 20
schedule_1250 2
schedule_1251 2
schedule_1252 2
schedule_1253 2
schedule_1254 2
schedule_1255 2
schedule_1256 2
schedule_1257 2
schedule_1258 2
schedule_1259 2
stay_1250 101
stay_1251 102
stay_1252 103
stay_1253 104
stay_1254 105
stay_1255 106
stay_1256 107
stay_1257 108
stay_1258 109
stay_1259 110
sat
1260 20
Ulimate Cost 1260 2283
Num_timeslot 10 Optimal_cost 20
schedule_1260 2
schedule_1261 2
schedule_1262 2
schedule_1263 2
schedule_1264 2
schedule_1265 2
schedule_1266 2
schedule_1267 2
schedule_1268 2
schedule_1269 2
stay_1260 111
stay_1261 112
stay_1262 113
stay_1263 114
stay_1264 115
stay_1265 116
stay_1266 117
stay_1267 118
stay_1268 119
stay_1269 120
sat
1270 20
Ulimate Cost 1270 2303
Num_timeslot 10 Optimal_cost 20
schedule_1270 2
schedule_1271 2
schedule_1272 2
schedule_1273 2
schedule_1274 2
schedule_1275 2
schedule_1276 2
schedule_1277 2
schedule_1278 2
schedule_1279 2
stay_1270 121
stay_1271 122
stay_1272 1

# House-B Occupant-2 Greedy Schedule

In [29]:
greedy_schedule_house_B_occ_2 = greedy_schedule(clusters_house_B_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/KMeans_House-B_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 60
selected zone 2
Here 60 61
selected zone 4
Here 61 62
selected zone 4
Here 62 63
selected zone 4
Here 63 64
selected zone 4
Here 64 65
selected zone 4
Here 65 66
selected zone 4
Here 66 67
selected zone 4
Here 67 68
selected zone 4
Here 68 69
selected zone 4
Here 69 70
selected zone 4
Here 70 71
selected zone 4
Here 71 72
selected zone 4
Here 72 73
selected zone 4
Here 73 74
selected zone 4
Here 74 75
selected zone 4
Here 75 76
selected zone 4
Here 76 77
selected zone 4
Here 77 78
selected zone 4
Here 78 79
selected zone 4
Here 79 80
selected zone 4
Here 80 81
selected zone 4
Here 81 82
selected zone 4
Here 82 83
selected zone 4
Here 83 84
selected zone 4
Here 84 85
selected zone 4
Here 85 86
selected zone 4
Here 86 87
selected zone 4
Here 87 88
selected zone 4
Here 88 89
selected zone 4
Here 89 90
selected zone 4
Here 90 91
selected zone 4
Here 91 92
selected zone 4
Here 92 93
selected zone 4
Here 93 94
selected zone 4
Here 94 95
selected zone 4
Here 95 96
selected zone 4
He